In [1]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import linear_model
import nltk
import random
import re
import nltk.corpus
import sklearn
import numpy
import csv
from sklearn import metrics
import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True) #se não ignorar os stopwords, não vai remover eles depois

#inicializando bag de stopwords
stopwords = nltk.corpus.stopwords.words('english')

import os
print(os.listdir("../input"))

['cleandata', 'jigsaw-unintended-bias-in-toxicity-classification']


In [2]:
base = []
csvFile = "../input/cleandata/cleanDataTrain.csv"

baseTrain = []
labels = []
allWords = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            temp1 = row[1]
            allWords.append(temp1)
            baseTrain.append(temp1)
            temp2 = float(row[0])
            labels.append(temp2)
        except IndexError:
            pass

In [3]:
baseTest = []
csvFile = "../input/cleandata/cleanDataTest.csv"
ids = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            # Se for pegar o valor float
            temp2 = row[0]
            temp1 = row[1]
            if(temp2!="id"):
                baseTest.append(temp1)
                ids.append(temp2)
        except IndexError:
            pass

In [4]:
x = baseTrain
y = labels

In [5]:
x_min = []
y_min = []
for u in range(len(y)):
    if(y[u]!=0):
        x_min.append(x[u])
        y_min.append(y[u])

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
#vectorizer_TF = CountVectorizer()
#X_TFIDF = vectorizer_TF.fit_transform(x_min)
#X_TestTFIDF = vectorizer_TF.transform(baseTest)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_TFIDF = TfidfVectorizer(min_df=0.001)
X_TFIDF = vectorizer_TFIDF.fit_transform(x_min)
X_TestTFIDF = vectorizer_TFIDF.transform(baseTest)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_min, test_size=0.10, random_state=42)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
#model.add(Dense(1000, activation='relu'))

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Classificação com probabilidade
y_train_class = []
for t in range(len(y_train)):
    if(y_train[t]<0.5):
        y_train_class.append(0)
    else:
        y_train_class.append(1)

model.fit(X_train, y_train_class, epochs=10, batch_size=1024)
scores = model.evaluate(X_train, y_train_class)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/10
486099/486099 [==============================] - 17s 35us/step - loss: 0.4317 - acc: 0.8102
Epoch 2/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.3676 - acc: 0.8412
Epoch 3/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.2062 - acc: 0.9175
Epoch 4/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.0712 - acc: 0.9748
Epoch 5/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.0438 - acc: 0.9849
Epoch 6/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.0341 - acc: 0.9875
Epoch 7/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.0299 - acc: 0.9885
Epoch 8/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.0270 - acc: 0.9894
Epoch 9/10
486099/486099 [==============================] - 16s 32us/step - loss: 0.0257 - acc: 0.9898
Epoch 10/10
486099/486099 [==============================] - 30s 61us/ste

In [15]:
from sklearn import metrics
#y_pred = clf.predict(X_test) 
y_pred = model.predict(X_test) 
y_pred_prob = y_pred
print(y_pred_prob)
print(X_test.shape)

y_pred_class = []
y_test_class = []
y_pred_prob_tox = []

for t in range(len(y_pred)):
    if(y_pred[t][0]<0.5):
        y_pred_class.append(0)
    else:
        y_pred_class.append(1)

for t in range(len(y_test)):
    if(y_test[t]<0.5):
        y_test_class.append(0)
    else:
        y_test_class.append(1)
#    y_pred_prob_tox.append(y_pred_prob[t][0])

print(y_test_class[0], y_pred_class[0])

mse = metrics.mean_squared_error(y_test, y_pred)
auc = metrics.roc_auc_score(y_test_class, y_pred_class)
acc = metrics.accuracy_score(y_test_class, y_pred_class)
prec = metrics.precision_score(y_test_class, y_pred_class)
rec = metrics.recall_score(y_test_class, y_pred_class)

print(auc)
print(acc)
print(prec)
print(rec)
print(mse)

with open('results_validation.csv', mode='w') as employee_file:
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Logistic Regression"])
    writer.writerow(["mse: ", mse])
    writer.writerow(["auc: ", auc])
    writer.writerow(["accuracy: ", acc])
    writer.writerow(["precision: ", prec])
    writer.writerow(["recall: ", rec])

[[1.9572204e-01]
 [9.9995512e-01]
 [2.6923418e-04]
 ...
 [4.0402937e-01]
 [5.6487853e-03]
 [1.6981879e-02]]
(54011, 3303)
1 0
0.7275158161436068
0.8018736923959934
0.6427501379799732
0.5691942466136014
0.12266996438292371


In [ ]:
y_pred_test = model.predict(X_TestTFIDF)
with open('submission.csv', mode='w') as employee_file:
    
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "prediction"])
    for id in range(len(ids)):
        print(ids[id])
        print(y_pred_test[id])
        writer.writerow([ids[id], y_pred_test[id][0]])